*A document in progress showing my scraper development.*

### Import modules
*(Always do this step before running any other code chunks)*

In [6]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import certifi
import urllib3
import pandas as pd
import numpy as np
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

## New listings (brands = lululemon, Reformation, Sezane, Doen, Realisation Par, Iro, Ulla Johnson, Zimmermann, The Row, Rouje)

Here, I specify the URL for **sold/completed** listings of 10 brands of women's clothing in **New** condition. (It's all in the link). Reminder to check the [robot.txt](https://www.ebay.com/robots.txt) for the site you want to scrape- don't get blocked as a bot!

I'm also identifying the page for BeautifulSoup.

#### Change the object below to correspond to the brand page I want to scrape...

In [99]:
#brand = 'lululemon'
#brand = 'Reformation'
#brand = 'Sezane'
#brand = 'Doen'
#brand = 'realisation+par'
#brand = 'iro'
#brand = 'ulla+johnson'
#brand = 'zimmermann'
#brand = 'the+row'
brand = 'rouje'

In [100]:
urlpage = 'https://www.ebay.com/sch/i.html?_nkw=' + str(brand) + '&LH_ItemCondition=1500|1000&LH_Complete=1&rt=nc&LH_Sold=1'
print(urlpage)

https://www.ebay.com/sch/i.html?_nkw=rouje&LH_ItemCondition=1500|1000&LH_Complete=1&rt=nc&LH_Sold=1


In [101]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
r = http.request('GET', urlpage)
page = urllib.request.urlopen(urlpage).read()
soup = BeautifulSoup(page, 'html.parser')
# print(page)

In [102]:
# For testing
item_containers = soup.find_all('div', {'class': 's-item__info clearfix'})
print(len(item_containers)) # should be about 4 dozen
# item_containers[0]

48


### Running the scraper
On this site, there are 48 listings per page. We are starting to scrape at page 1; I want to scrape 500 pages. I'm retrieving 1) title/item details in a messy HTML chunk, to be cleaned later; 2) price.

In [103]:
page_num = 1

# Create lists to store the scraped information
summary = []
price = []

while page_num<=500:
    html = requests.get(urlpage.format(page_num)).text
    soup = BeautifulSoup(html, 'html.parser')

    for item in soup.select('div.s-item__info.clearfix'):
        if item.select_one("h3.s-item__title"):
            summary.append(item.select_one("h3.s-item__title").text)
        if item.select_one("span.s-item__price"):
            price.append(item.select_one("span.s-item__price").text)

    page_num=page_num+1

# Check the results
print(len(summary))
print(len(price))

24000
24000


### Storing the data in a pandas dataframe (change this for each brand run-through)

In [106]:
new_eBay_rouje_df = pd.DataFrame({'summary': summary, 'price': price})
print(new_eBay_rouje_df.info())
new_eBay_rouje_df.head()

# Ew that looks like a lot of memory usage but I'll, uh, deal with it later

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 2 columns):
summary    24000 non-null object
price      24000 non-null object
dtypes: object(2)
memory usage: 375.1+ KB
None


,summary,price
0,ROUJE Monique Dress Blue Watercolor Silk Size ...,$221.00
1,ROUJE PALOMA Dress Navy Flower Polka Size 38 O...,$198.00
2,ROUJE Jose Blouse Red Jonquil Size 40 Orig. $1...,$128.00
3,ROUJE Gabin Dress Light Blue Floral Print Size...,$190.00
4,Rouje NWT Sold Out Angèle skirt Size 34/XS&S,$160.00


## Pre-owned listings

Here's the workflow again, with less commentary. In this instance, I am applying the scraper to retrieve **sold/completed** listings of 10 brands of women's clothing in **Pre-owned** condition.

#### As above, change the object below to correspond to the brand I'm scraping.

In [107]:
#brand = 'lululemon'
#brand = 'Reformation'
#brand = 'Sezane'
#brand = 'Doen'
#brand = 'realisation+par'
#brand = 'iro'
#brand = 'ulla+johnson'
#brand = 'zimmermann'
#brand = 'the+row'
brand = 'rouje'

In [108]:
urlpage = 'https://www.ebay.com/sch/i.html?_nkw=' + str(brand) + '&LH_Complete=1&LH_Sold=1&rt=nc&LH_ItemCondition=3000'
print(urlpage)

https://www.ebay.com/sch/i.html?_nkw=rouje&LH_Complete=1&LH_Sold=1&rt=nc&LH_ItemCondition=3000


In [109]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
r = http.request('GET', urlpage)
page = urllib.request.urlopen(urlpage).read()
soup = BeautifulSoup(page, 'html.parser')

In [111]:
page_num = 1

# Create lists to store the scraped information
summary = []
price = []

while page_num<=500:
    html = requests.get(urlpage.format(page_num)).text
    soup = BeautifulSoup(html, 'html.parser')

    for item in soup.select('div.s-item__info.clearfix'):
        if item.select_one("h3.s-item__title"):
            summary.append(item.select_one("h3.s-item__title").text)
        if item.select_one("span.s-item__price"):
            price.append(item.select_one("span.s-item__price").text)

    page_num=page_num+1

# Check the results
print(len(summary))
print(len(price))

SSLError: HTTPSConnectionPool(host='www.ebay.com', port=443): Max retries exceeded with url: /sch/i.html?_nkw=rouje&LH_Complete=1&LH_Sold=1&rt=nc&LH_ItemCondition=3000 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')",),))

In [112]:
print(len(summary))
print(len(price))

11088
11088


#### Change the df name each time, an unfortunate hassle that avoids confusion...

In [113]:
po_eBay_rouje_df = pd.DataFrame({'summary': summary, 'price': price})
print(po_eBay_rouje_df.info())
po_eBay_rouje_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11088 entries, 0 to 11087
Data columns (total 2 columns):
summary    11088 non-null object
price      11088 non-null object
dtypes: object(2)
memory usage: 173.4+ KB
None


,summary,price
0,Rouje Green Cardigan Size 34 XS As Seen On Jea...,$120.00
1,ROUJE Cream Wool Roundneck Cardigan Size 36,$125.73
2,"Rouje Yasmine Top, Size 36, Like New, SOLD OUT...",$60.29
3,Rouje Giulia T shirt 34,$40.00
4,Rouje (Jeanne Demas) ‘Remi’ T-shirt - Size 40 ...,$15.63


Run through all 10 brands before proceeding below.

As proof-of-concept, I created this workflow using the first two brands, so the code below referring to those data is commented out.

## Cleaning the data

Here is a list of my dfs:

In [105]:
'''
#print(new_eBay_lululemon_df.head())
#print(po_eBay_lululemon_df.head())
#print(new_eBay_ref_df.head())
#print(po_eBay_ref_df.head())
print(new_eBay_sezane_df.head())
print(po_eBay_sezane_df.head())
print(new_eBay_doen_df.head())
print(po_eBay_doen_df.head())
print(new_eBay_realisationpar_df.head())
print(po_eBay_realisationpar_df.head())
print(new_eBay_iro_df.head())
print(po_eBay_iro_df.head())
print(new_eBay_ullajohnson_df.head())
print(po_eBay_ullajohnson_df.head())
print(new_eBay_zimmermann_df.head())
print(po_eBay_zimmermann_df.head())
print(new_eBay_therow_df.head())
print(po_eBay_therow_df.head())
print(new_eBay_rouje_df.head())
print(po_eBay_rouje_df.head())
'''

'\n#print(new_eBay_lululemon_df.head())\n#print(po_eBay_lululemon_df.head())\n#print(new_eBay_ref_df.head())\n#print(po_eBay_ref_df.head())\nprint(new_eBay_sezane_df.head())\nprint(po_eBay_sezane_df.head())\nprint(new_eBay_doen_df.head())\nprint(po_eBay_doen_df.head())\nprint(new_eBay_realisationpar_df.head())\nprint(po_eBay_realisationpar_df.head())\nprint(new_eBay_iro_df.head())\nprint(po_eBay_iro_df.head())\nprint(new_eBay_ullajohnson_df.head())\nprint(po_eBay_ullajohnson_df.head())\nprint(new_eBay_zimmermann_df.head())\nprint(po_eBay_zimmermann_df.head())\nprint(new_eBay_therow_df.head())\nprint(po_eBay_therow_df.head())\n'

#### Adding new columns: brand, condition, category

In [114]:
#new_eBay_lululemon_df['brand'] = 'lululemon'
#po_eBay_lululemon_df['brand'] = 'lululemon'
#new_eBay_ref_df['brand'] = 'reformation'
#po_eBay_ref_df['brand'] = 'reformation'
new_eBay_sezane_df['brand'] = 'sezane'
po_eBay_sezane_df['brand'] = 'sezane'
new_eBay_doen_df['brand'] = 'doen'
po_eBay_doen_df['brand'] = 'doen'
new_eBay_realisationpar_df['brand'] = 'realisationpar'
po_eBay_realisationpar_df['brand'] = 'realisationpar'
new_eBay_iro_df['brand'] = 'iro'
po_eBay_iro_df['brand'] = 'iro'
new_eBay_ullajohnson_df['brand'] = 'ullajohnson'
po_eBay_ullajohnson_df['brand'] = 'ullajohnson'
new_eBay_zimmermann_df['brand'] = 'zimmermann'
po_eBay_zimmermann_df['brand'] = 'zimmermann'
new_eBay_therow_df['brand'] = 'therow'
po_eBay_therow_df['brand'] = 'therow'
new_eBay_rouje_df['brand'] = 'rouje'
po_eBay_rouje_df['brand'] = 'rouje'

#new_eBay_lululemon_df['condition'] = 'new'
#po_eBay_lululemon_df['condition'] = 'preowned'
#new_eBay_ref_df['condition'] = 'new'
#po_eBay_ref_df['condition'] = 'preowned'
new_eBay_sezane_df['condition'] = 'new'
po_eBay_sezane_df['condition'] = 'preowned'
new_eBay_doen_df['condition'] = 'new'
po_eBay_doen_df['condition'] = 'preowned'
new_eBay_realisationpar_df['condition'] = 'new'
po_eBay_realisationpar_df['condition'] = 'preowned'
new_eBay_iro_df['condition'] = 'new'
po_eBay_iro_df['condition'] = 'preowned'
new_eBay_ullajohnson_df['condition'] = 'new'
po_eBay_ullajohnson_df['condition'] = 'preowned'
new_eBay_zimmermann_df['condition'] = 'new'
po_eBay_zimmermann_df['condition'] = 'preowned'
new_eBay_therow_df['condition'] = 'new'
po_eBay_therow_df['condition'] = 'preowned'
new_eBay_rouje_df['condition'] = 'new'
po_eBay_rouje_df['condition'] = 'preowned'

#new_eBay_lululemon_df['category'] = ''
#po_eBay_lululemon_df['category'] = ''
#new_eBay_ref_df['category'] = ''
#po_eBay_ref_df['category'] = ''
new_eBay_sezane_df['category'] = ''
po_eBay_sezane_df['category'] = ''
new_eBay_doen_df['category'] = ''
po_eBay_doen_df['category'] = ''
new_eBay_realisationpar_df['category'] = ''
po_eBay_realisationpar_df['category'] = ''
new_eBay_iro_df['category'] = ''
po_eBay_iro_df['category'] = ''
new_eBay_ullajohnson_df['category'] = ''
po_eBay_ullajohnson_df['category'] = ''
new_eBay_zimmermann_df['category'] = ''
po_eBay_zimmermann_df['category'] = ''
new_eBay_therow_df['category'] = ''
po_eBay_therow_df['category'] = ''
new_eBay_rouje_df['category'] = ''
po_eBay_rouje_df['category'] = ''

#### Convert all 'summary' strings to lowercase

In [115]:
#new_eBay_lululemon_df['summary']= new_eBay_lululemon_df['summary'].str.lower()
#po_eBay_lululemon_df['summary'] = po_eBay_lululemon_df['summary'].str.lower()
#new_eBay_ref_df['summary']= new_eBay_ref_df['summary'].str.lower()
#po_eBay_ref_df['summary'] = po_eBay_ref_df['summary'].str.lower()
new_eBay_sezane_df['summary'] = new_eBay_sezane_df['summary'].str.lower()
po_eBay_sezane_df['summary'] = po_eBay_sezane_df['summary'].str.lower()
new_eBay_doen_df['summary'] = new_eBay_doen_df['summary'].str.lower()
po_eBay_doen_df['summary'] = po_eBay_doen_df['summary'].str.lower()
new_eBay_realisationpar_df['summary'] = new_eBay_realisationpar_df['summary'].str.lower()
po_eBay_realisationpar_df['summary'] = po_eBay_realisationpar_df['summary'].str.lower()
new_eBay_iro_df['summary'] = new_eBay_iro_df['summary'].str.lower()
po_eBay_iro_df['summary'] = po_eBay_iro_df['summary'].str.lower()
new_eBay_ullajohnson_df['summary'] = new_eBay_ullajohnson_df['summary'].str.lower()
po_eBay_ullajohnson_df['summary'] = po_eBay_ullajohnson_df['summary'].str.lower()
new_eBay_zimmermann_df['summary'] = new_eBay_zimmermann_df['summary'].str.lower()
po_eBay_zimmermann_df['summary'] = po_eBay_zimmermann_df['summary'].str.lower()
new_eBay_therow_df['summary'] = new_eBay_therow_df['summary'].str.lower()
po_eBay_therow_df['summary'] = po_eBay_therow_df['summary'].str.lower()
new_eBay_rouje_df['summary'] = new_eBay_rouje_df['summary'].str.lower()
po_eBay_rouje_df['summary'] = po_eBay_rouje_df['summary'].str.lower()

#### Assigning the items to categories based on keywords pulled from the summary description

In [134]:
tops = ['top', 'shirt', 'tee', 'tank']
bottoms = ['skirt', 'pants', 'pant', 'short', 'crop', 'crops', 'shorts', 'leggings', 'tights', 'jeans']
dresses = ['dress']
#jumpsuits = ['jumpsuit']
bras = ['bra', 'bras']
outerwear = ['jacket', 'hoodie', 'sweater', 'sweatshirt', 'rainjacket', 'coat', 'parka', 'turtleneck']
#accessories = ['headband', 'bag', 'beanie', 'hat', 'toque', 'scarf', 'gloves', 'socks']
#bag = ['backpack', 'tote', 'duffel']

def fun(a):
    for i in tops:
        if i in a:
            return "tops"
    for j in bottoms:
        if j in a:
            return "bottoms"
    for k in bras:
        if k in a:
            return "bras"
    for l in dresses:
        if l in a:
            return "dresses"
    for m in outerwear:
        if m in a:
            return "outerwear"
    return "other"

#new_eBay_lululemon_df.category = new_eBay_lululemon_df.summary.apply(lambda x: fun(x))
#po_eBay_lululemon_df.category = po_eBay_lululemon_df.summary.apply(lambda x: fun(x))
#new_eBay_ref_df.category = new_eBay_ref_df.summary.apply(lambda x: fun(x))
#po_eBay_ref_df.category = po_eBay_ref_df.summary.apply(lambda x: fun(x))
new_eBay_sezane_df.category = new_eBay_sezane_df.summary.apply(lambda x: fun(x))
po_eBay_sezane_df.category = po_eBay_sezane_df.summary.apply(lambda x: fun(x))
new_eBay_doen_df.category = new_eBay_doen_df.summary.apply(lambda x: fun(x))
po_eBay_doen_df.category = po_eBay_doen_df.summary.apply(lambda x: fun(x))
new_eBay_realisationpar_df.category = new_eBay_realisationpar_df.summary.apply(lambda x: fun(x))
po_eBay_realisationpar_df.category = po_eBay_realisationpar_df.summary.apply(lambda x: fun(x))
new_eBay_iro_df.category = new_eBay_iro_df.summary.apply(lambda x: fun(x))
po_eBay_iro_df.category = po_eBay_iro_df.summary.apply(lambda x: fun(x))
new_eBay_ullajohnson_df.category = new_eBay_ullajohnson_df.summary.apply(lambda x: fun(x))
po_eBay_ullajohnson_df.category = po_eBay_ullajohnson_df.summary.apply(lambda x: fun(x))
new_eBay_zimmermann_df.category = new_eBay_zimmermann_df.summary.apply(lambda x: fun(x))
po_eBay_zimmermann_df.category = po_eBay_zimmermann_df.summary.apply(lambda x: fun(x))
new_eBay_therow_df.category = new_eBay_therow_df.summary.apply(lambda x: fun(x))
po_eBay_therow_df.category = po_eBay_therow_df.summary.apply(lambda x: fun(x))
new_eBay_rouje_df.category = new_eBay_rouje_df.summary.apply(lambda x: fun(x))
po_eBay_rouje_df.category = po_eBay_rouje_df.summary.apply(lambda x: fun(x))

#### Strip dollar sign from 'price'

In [117]:
#new_eBay_lululemon_df['price'] = new_eBay_lululemon_df['price'].str.replace('$', '')
#po_eBay_lululemon_df['price'] = po_eBay_lululemon_df['price'].str.replace('$', '')
#new_eBay_ref_df['price'] = new_eBay_ref_df['price'].str.replace('$', '')
#po_eBay_ref_df['price'] = po_eBay_ref_df['price'].str.replace('$', '')
new_eBay_sezane_df['price'] = new_eBay_sezane_df['price'].str.replace('$', '')
po_eBay_sezane_df['price'] = po_eBay_sezane_df['price'].str.replace('$', '')
new_eBay_doen_df['price'] = new_eBay_doen_df['price'].str.replace('$', '')
po_eBay_doen_df['price'] = po_eBay_doen_df['price'].str.replace('$', '')
new_eBay_realisationpar_df['price'] = new_eBay_realisationpar_df['price'].str.replace('$', '')
po_eBay_realisationpar_df['price'] = po_eBay_realisationpar_df['price'].str.replace('$', '')
new_eBay_iro_df['price'] = new_eBay_iro_df['price'].str.replace('$', '')
po_eBay_iro_df['price'] = po_eBay_iro_df['price'].str.replace('$', '')
new_eBay_ullajohnson_df['price'] = new_eBay_ullajohnson_df['price'].str.replace('$', '')
po_eBay_ullajohnson_df['price'] = po_eBay_ullajohnson_df['price'].str.replace('$', '')
new_eBay_zimmermann_df['price'] = new_eBay_zimmermann_df['price'].str.replace('$', '')
po_eBay_zimmermann_df['price'] = po_eBay_zimmermann_df['price'].str.replace('$', '')
new_eBay_therow_df['price'] = new_eBay_therow_df['price'].str.replace('$', '')
po_eBay_therow_df['price'] = po_eBay_therow_df['price'].str.replace('$', '')
new_eBay_rouje_df['price'] = new_eBay_rouje_df['price'].str.replace('$', '')
po_eBay_rouje_df['price'] = po_eBay_rouje_df['price'].str.replace('$', '')

#### Add a site column and save as CSV for now

In [138]:
#new_eBay_lululemon_df['site'] = 'ebay'
#po_eBay_lululemon_df['site'] = 'ebay'
#new_eBay_ref_df['site'] = 'ebay'
#po_eBay_ref_df['site'] = 'ebay'
new_eBay_sezane_df['site'] = 'ebay'
po_eBay_sezane_df['site'] = 'ebay'
new_eBay_doen_df['site'] = 'ebay'
po_eBay_doen_df['site'] = 'ebay'
new_eBay_realisationpar_df['site'] = 'ebay'
po_eBay_realisationpar_df['site'] = 'ebay'
new_eBay_iro_df['site'] = 'ebay'
po_eBay_iro_df['site'] = 'ebay'
new_eBay_ullajohnson_df['site'] = 'ebay'
po_eBay_ullajohnson_df['site'] = 'ebay'
new_eBay_zimmermann_df['site'] = 'ebay'
po_eBay_zimmermann_df['site'] = 'ebay'
new_eBay_therow_df['site'] = 'ebay'
po_eBay_therow_df['site'] = 'ebay'
new_eBay_rouje_df['site'] = 'ebay'
po_eBay_rouje_df['site'] = 'ebay'


#pd.DataFrame.to_csv(new_eBay_lululemon_df, 'new_eBay_lululemon_df.csv') # Can also save to .json
#pd.DataFrame.to_csv(po_eBay_lululemon_df, 'po_eBay_lululemon_df.csv')
#pd.DataFrame.to_csv(new_eBay_ref_df, 'new_eBay_ref_df.csv')
#pd.DataFrame.to_csv(po_eBay_ref_df, 'po_eBay_ref_df.csv')
pd.DataFrame.to_csv(new_eBay_sezane_df, 'new_eBay_sezane_df.csv') 
pd.DataFrame.to_csv(po_eBay_sezane_df, 'po_eBay_sezane_df.csv')
pd.DataFrame.to_csv(new_eBay_doen_df, 'new_eBay_doen_df.csv')
pd.DataFrame.to_csv(po_eBay_doen_df, 'po_eBay_doen_df.csv')
pd.DataFrame.to_csv(new_eBay_realisationpar_df, 'new_eBay_realisationpar_df.csv')
pd.DataFrame.to_csv(po_eBay_realisationpar_df, 'po_eBay_realisationpar_df.csv')
pd.DataFrame.to_csv(new_eBay_iro_df, 'new_eBay_iro_df.csv')
pd.DataFrame.to_csv(po_eBay_iro_df, 'po_eBay_iro_df.csv')
pd.DataFrame.to_csv(new_eBay_ullajohnson_df, 'new_eBay_ullajohnson_df.csv')
pd.DataFrame.to_csv(po_eBay_ullajohnson_df, 'po_eBay_ullajohnson_df.csv')
pd.DataFrame.to_csv(new_eBay_zimmermann_df, 'new_eBay_zimmermann_df.csv')
pd.DataFrame.to_csv(po_eBay_zimmermann_df, 'po_eBay_zimmermann_df.csv')
pd.DataFrame.to_csv(new_eBay_therow_df, 'new_eBay_therow_df.csv')
pd.DataFrame.to_csv(po_eBay_therow_df, 'po_eBay_therow_df.csv')
pd.DataFrame.to_csv(new_eBay_rouje_df, 'new_eBay_rouje_df.csv')
pd.DataFrame.to_csv(po_eBay_rouje_df, 'po_eBay_rouje_df.csv')